<a href="https://colab.research.google.com/github/Douglas1997Ortiz/predicciones-de-ventas/blob/main/Proyecto_Parte5_Corregidoipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import set_config


In [132]:
import sklearn 
print(sklearn.__version__)


1.2.2


In [133]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [134]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
dfventas= pd.read_csv('/content/drive/MyDrive/Coding Dojo/PROYECTO 1/sales_predictions.csv')
dfventas.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [136]:
dfventas.shape

(8523, 12)

In [137]:
dfventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [138]:
dfventas.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [139]:
# Identifiquen el objetivo (X) y las características (y): Asignen la columna "Item_Outlet_Sales" somo el objetivo y el resto de las variables relevantes como el matriz de características. 
X = dfventas.drop('Item_Outlet_Sales', axis=1)
y = dfventas['Item_Outlet_Sales']

In [140]:
# Realicen un train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [141]:
# Creen un pipeline de preprocesamiento para preparar el conjunto de datos para el aprendizaje automático.

In [142]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [143]:
# Se utiliza la imputacion de moda en la variable del tamano de los locales 
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [144]:
# Numeric pipeline
numeric_pipe = make_pipeline(num_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [145]:
# Categorical pipeline
categorical_pipe = make_pipeline(cat_imputer)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent'))])

In [146]:
# Instanciar ColumnTransformer
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fbfc9e501c0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fbfc9d4deb0>)])

In [147]:
# Transformador de datos
# fit on train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fbfc9e501c0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fbfc9d4deb0>)])

In [148]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [149]:
print(X_train_processed)

[[0.8172486782467712 -0.7127750716052249 1.8281092189993307 ... 'Medium'
  'Tier 3' 'Supermarket Type2']
 [0.5563395034619393 -1.291052247079188 0.6033688805132759 ... 'Medium'
  'Tier 3' 'Supermarket Type2']
 [-0.131511957334435 1.8133186433439548 0.24454055715071762 ... 'Medium'
  'Tier 1' 'Supermarket Type1']
 ...
 [1.113736376865898 -0.9205271269932742 1.5230267401668727 ... 'Medium'
  'Tier 2' 'Supermarket Type1']
 [1.7660093138279769 -0.22775520135060978 -0.38377707627087376 ...
  'Medium' 'Tier 2' 'Supermarket Type1']
 [0.8172486782467712 -0.9586768265695343 -0.7383610459950399 ... 'Small'
  'Tier 1' 'Supermarket Type1']]


In [150]:
print(X_test_processed)

[[0.33100885251140305 -0.7766462476459127 -0.9988155355551928 ... 'High'
  'Tier 3' 'Supermarket Type1']
 [-1.1798924596516673 0.10031660276043455 -1.5851942284354996 ... 'Small'
  'Tier 1' 'Supermarket Type1']
 [0.3784468842904632 -0.4829943187611561 -1.5957843538417513 ... 'Medium'
  'Tier 1' 'Supermarket Type1']
 ...
 [-1.139570132639466 1.2183242782860708 1.0939797535632823 ... 'Medium'
  'Tier 2' 'Supermarket Type1']
 [-1.4977272725713713 -0.7780956710837439 -0.36679966171730477 ...
  'Medium' 'Tier 1' 'Supermarket Type1']
 [0.520760979627644 -0.779762926606579 0.11221189424117377 ... 'Medium'
  'Tier 2' 'Supermarket Type1']]
